In [1]:
# Bag of Words Meets Bags of Popcorn

!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c sf-crime

!kaggle competitions download -c word2vec-nlp-tutorial # Rules에서 허락 필요

!unzip labeledTrainData.tsv.zip
!unzip unlabeledTrainData.tsv.zip
!unzip testData.tsv.zip

import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import callbacks, models, layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train = pd.read_csv('/content/labeledTrainData.tsv',sep='\t')
unlabeledtrain = pd.read_csv('/content/unlabeledTrainData.tsv',sep='\t', error_bad_lines=False)
test = pd.read_csv('/content/testData.tsv',sep='\t')
sampleSubmission = pd.read_csv('/content/sampleSubmission.csv')

train.head()

Saving kaggle.json to kaggle.json
 27% 6.00M/22.1M [00:00<00:00, 62.5MB/s]
100% 22.1M/22.1M [00:00<00:00, 141MB/s] 
 53% 10.0M/18.7M [00:00<00:00, 103MB/s]
100% 18.7M/18.7M [00:00<00:00, 119MB/s]
  0% 0.00/2.35M [00:00<?, ?B/s]
100% 2.35M/2.35M [00:00<00:00, 161MB/s]
 93% 12.0M/13.0M [00:00<00:00, 125MB/s]
100% 13.0M/13.0M [00:00<00:00, 120MB/s]
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 89.6MB/s]
 65% 17.0M/26.0M [00:00<00:00, 61.5MB/s]
100% 26.0M/26.0M [00:00<00:00, 74.7MB/s]
  0% 0.00/12.6M [00:00<?, ?B/s]
100% 12.6M/12.6M [00:00<00:00, 116MB/s]
Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    
Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  
Archive:  testData.tsv.zip
  inflating: testData.tsv            
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 43043: expected 2 fields, saw 3\n'


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [2]:
# 전처리
stop_word = stopwords.words('english')

def clean (data):
  data = BeautifulSoup(data).get_text() # <br> 같은 html 용어 제거
  data = re.sub("[^a-zA-Z]", " ", data) # 문자 아닌 것 제거
  data = data.lower().split() # 소문자, 벡터화
  data = [i for i in data if i not in stop_word]
  return ' '.join(data)

train["cleaned_review"] = train["review"].apply(clean) # 각 행에 함수 적용

In [3]:
tokenizer = Tokenizer(num_words=6000)
tokenizer.fit_on_texts(train.cleaned_review)

sequences = tokenizer.texts_to_sequences(train.cleaned_review) # 단어 개수
max_ = max([len(x) for x in sequences])
padded_sequences = np.array(pad_sequences(sequences, maxlen=max_, padding='pre')) # 길이 채움 : 패딩

X_train, X_val, y_train, y_val = train_test_split(padded_sequences, np.array(train.sentiment), test_size=0.2, random_state=0)

In [4]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPool1D, Dropout

model = Sequential()

model.add(Embedding(input_dim=6000, output_dim=64))
model.add(LSTM(32, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 15
validation_split = 0.01
model.fit(x=padded_sequences, y=np.array(train.sentiment), batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/15
774/774 [==============================] - 24s 29ms/step - loss: 0.0210 - accuracy: 0.9933 - val_loss: 0.6299 - val_accuracy: 0.8760
Epoch 2/15
774/774 [==============================] - 22s 28ms/step - loss: 0.0138 - accuracy: 0.9966 - val_loss: 0.6396 - val_accuracy: 0.8840
Epoch 3/15
774/774 [==============================] - 22s 28ms/step - loss: 0.0104 - accuracy: 0.9977 - val_loss: 0.7773 - val_accuracy: 0.8800
Epoch 4/15
774/774 [==============================] - 22s 28ms/step - loss: 0.0091 - accuracy: 0.9976 - val_loss: 0.7639 - val_accuracy: 0.8800
Epoch 5/15
774/774 [==============================] - 22s 28ms/step - loss: 0.0101 - accuracy: 0.9975 - val_loss: 0.7504 - val_accuracy: 0.8920
Epoch 6/15
774/774 [==============================] - 22s 28ms/step - loss: 0.0134 - accuracy: 0.9966 - val_loss: 0.7534 - val_accuracy: 0.8720
Epoch 7/15
774/774 [==============================] - 22s 28ms/step - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.8805 - val_accuracy:

In [7]:
test["cleaned_review"] = test["review"].apply(clean)

sequence = tokenizer.texts_to_sequences(test.cleaned_review) # 단어 개수
ma = max([len(x) for x in sequences])
padded_ = np.array(pad_sequences(sequence, maxlen=ma, padding='pre'))

pred = model.predict(x=padded_)

In [8]:
y_pred = (pred >= 0.5) * 1
results = pd.DataFrame({"id": test.id, "sentiment": y_pred.flatten()})
results.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,1


In [9]:
results.to_csv("submission.csv", index=False)

In [10]:
!kaggle competitions submit -c word2vec-nlp-tutorial -f submission.csv -m " "

100% 227k/227k [00:00<00:00, 379kB/s]
Successfully submitted to Bag of Words Meets Bags of Popcorn